# Analyzing the gradients of the SHT

## Setup

In [ ]:
import math
import numpy as np
import paddle
import paddle.nn as nn

from paddle_harmonics.quadrature import legendre_gauss_weights, clenshaw_curtiss_weights
from paddle_harmonics.legendre import legpoly, clm
from paddle_harmonics import RealSHT, InverseRealSHT

## Problem setting

We consider the simple problem of fitting the spectral coefficients $\theta$ such that

$$
\begin{align}
\mathcal{L}
&= ||\mathop{\mathrm{ISHT}}[\theta] - u^*||^2_{S^2} \\
&\approx \sum_j \omega_j (\mathop{\mathrm{ISHT}}[\theta](x_j) - u^*(x_j))^2 \\
&= (S \, \theta - u^*)^T \mathop{\mathrm{diag}}(\omega) \, (S \, \theta - u^*) \\
&= L
\end{align}
$$

is minimized.

The Vandermonde matrix $S$, which is characterized by $\mathop{\mathrm{ISHT}}[\theta] = S \theta$ realizes the action of the discrete SHT.

The necessary condition for a minimizer of $L$ is

$$
\begin{align}
& \nabla_\theta L = S^T \mathop{\mathrm{diag}}(\omega) \, (S \, \theta - u^*) = 0 \\
\Leftrightarrow \quad & S^T \mathop{\mathrm{diag}}(\omega) \, S \; \theta = S^T \mathop{\mathrm{diag}}(\omega) \, u^*.
\end{align}
$$

On the Gaussian grid, 

In [ ]:
nlat = 64
nlon = 2*nlat
grid = "equiangular"

# for quadrature and plotting
if grid == "legendre-gauss":
    lmax = mmax = nlat
    xq, wq = legendre_gauss_weights(nlat)
elif grid =="equiangular":
    lmax = mmax = nlat//2
    xq, wq = clenshaw_curtiss_weights(nlat)

sht = RealSHT(nlat, nlon, lmax=lmax, mmax=mmax, grid=grid)
isht = InverseRealSHT(nlat, nlon, lmax=lmax, mmax=mmax, grid=grid)

lat = np.arccos(xq)
omega = math.pi * paddle.to_tensor(wq).to(paddle.float32) / nlat
omega = omega.reshape([-1, 1])

nlon*omega.sum()

In [ ]:
!mkdir -p ./data
!wget https://astropedia.astrogeology.usgs.gov/download/Mars/GlobalSurveyor/MOLA/thumbs/Mars_MGS_MOLA_DEM_mosaic_global_1024.jpg -O ./data/mola_topo.jpg

In [ ]:
import imageio.v3 as iio

img = iio.imread('./data/mola_topo.jpg')
#convert to grayscale
data = np.dot(img[...,:3]/255, [0.299, 0.587, 0.114])
# interpolate onto 512x1024 grid:
data = nn.functional.interpolate(paddle.to_tensor(data).unsqueeze(0).unsqueeze(0), size=(nlat,nlon)).squeeze()

In [ ]:
import matplotlib.pyplot as plt
from plotting import plot_sphere

plot_sphere(data, cmap="turbo", colorbar=True)

In [ ]:
lr = 1.0
theta = paddle.create_parameter([lmax,lmax,2], paddle.float32, 
                                default_initializer=paddle.nn.initializer.Normal(mean=0, std=1))
optim = paddle.optimizer.SGD(parameters=[theta], learning_rate=lr)

for iter in range(40):
    optim.clear_grad(set_to_zero=True)
    loss = paddle.sum(0.5*omega*(isht(theta.as_complex()) - data)**2)
    loss.backward()

    # action of the Hessian
    with paddle.no_grad():
        for m in range(1,mmax):
            theta.grad[:,m].multiply_(paddle.to_tensor([0.5],dtype=paddle.float32))

    optim.step()

    print(f"iter: {iter}, loss: {loss}")

what's the best possible loss? $\theta^* = (S^T \mathop{\mathrm{diag}}(\omega) \, S)^{-1} S^T \mathop{\mathrm{diag}}(\omega) u^* = \mathop{\mathrm{SHT}}[u^*]$ gives us the global minimizer for this problem.

In [ ]:
fig = plt.figure(layout='constrained', figsize=(20, 12))
subfigs = fig.subfigures(2, 3)

# spectral fitting
plot_sphere(isht(theta.as_complex()).detach(), fig=subfigs[0,0], cmap="turbo", colorbar=True, title="Fit")
plot_sphere(data, fig=subfigs[0,1], cmap="turbo", colorbar=True, title="Ground truth")
plot_sphere((isht(theta.as_complex()) - data).detach(), fig=subfigs[0,2], cmap="turbo", colorbar=True, title="residual")

# sht(u)
plot_sphere(isht(sht(data)).detach(), fig=subfigs[1,0], cmap="turbo", colorbar=True, title="isht(sht(u))")
plot_sphere(data, fig=subfigs[1,1], cmap="turbo", colorbar=True, title="Ground truth")
plot_sphere((isht(sht(data)) - data).detach(), fig=subfigs[1,2], cmap="turbo", colorbar=True, title="residual")